In [9]:
using JuMP, Ipopt, Plots, Printf, LinearAlgebra, SCS, COSMO

N = 2
λ = 1
ni = 4 * (N + 1)^2
np = 5 * N + 1
ns  = ni - 12
nt = np + ns + 1

S = Dict()
T = []
TG = []
TB = []
for ua in 0:N
    for ub in 0:N
        for sa in 0:1
            for sb in 0:1
                index = (2 * sa + sb) * (N + 1)^2 + (N + 1) * ua + ub
                S[index] = (ua, ub, sa, sb)
            end
        end
    end
end
Skeyer = Dict(value => key for (key, value) in S);

T = []
TG = []
TB = []

# Loop through the ranges
for ua in 0:N
    for ub in 0:N
        for sa in 0:1
            for sb in 0:1
                i = (2 * sa + sb) * (N + 1)^2 + (N + 1) * ua + ub
                if ua == N && ub == 0
                    push!(TG, i)
                    push!(T, i)
                end
                if ua == 0 && ub == N
                    push!(TG, i)
                    push!(T, i)
                end
                if ua == N && ub == N
                    push!(TB, i)
                    push!(T, i)
                end
                S[i] = (ua, ub, sa, sb)
            end
        end
    end
end
Tc = [i for i in 0:ni-1 if i ∉ T]

sort!(T);
sort!(TG);
sort!(TB);

M=zeros(Int, ni, ni, np);
for (u, u_) in Iterators.product(values(S), values(S))
    # Skip if u and u_ are the same
    if u == u_
        continue
    end

    i = Skeyer[u]
    j = Skeyer[u_]
    flag = 0
    ua, ub, sa, sb = u
    ua_, ub_, sa_, sb_ = u_

    tempk = 0

    if j == i + (N + 1) && ub == ub_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = N * sa + ua + 1
    elseif j == i - (N + 1) && ub == ub_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = sa * N + ua + 2 * N 
    elseif j == i + 1 && ua == ua_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = N * sb + ub + 1
    elseif j == i - 1 && ua == ua_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = sb * N + ub + 2 * N 
    elseif j == i + 2 * (N + 1)^2 && ua == ua_ && ub == ub_ && ub != 0 && sb == sb_
        flag = 1
        tempk = 4 * N + ub 
    elseif j == i - 2 * (N + 1)^2 && ua == ua_ && ub == ub_ && sb == sb_
        flag = 1
        tempk = 5 * N + 1
    elseif j == i + (N + 1)^2 && ua == ua_ && ub == ub_ && ua != 0 && sa == sa_
        flag = 1
        tempk = 4 * N + ua 
    elseif j == i - (N + 1)^2 && ua == ua_ && ub == ub_ && sa == sa_
        flag = 1
        tempk = 5 * N + 1
    end

    if flag == 1
        M[i+1, j+1, tempk] = flag
    end
end
for k in 1:np
    # Calculate the sum of each row for the kth slice of M (equivalent to M[:,:,k] in Python)
    M_i = [sum(M[:, :, k][i, :]) for i in 1:ni]

    for u in values(S)
        i = Skeyer[u]
        M[i+1, i+1, k] = -M_i[i+1]
    end
end

R = zeros(ns, ni)
for i in 1:ns
    R[i,Tc[i]+1] = 1
end

Mtilde = zeros(ns, ns, np)
for k in 1:np
    Mtilde[:, :, k] = R * M[:, :, k] * R'
end

eis = Matrix{Float64}(I, ns, ns)
Ai = zeros(ns, np, ns)
for k in 1:ns
    ei=eis[k,:]
    for i in 1:ns
        for j in 1:np
            Ai[i,j,k]=ei'* Mtilde[:,i,j]   
        end 
    end
end

alpha=zeros(np,ns)
for i in 1:ns
    Si=Tc[i]
    for j in 1:ni
        for k in 1:np
            if (j-1 ∈ TB)
                alpha[k,i]+=M[Si+1,j,k]
            end
            
        end
    end
end

Di=zeros(nt,nt,ns)
for i in 1:ns
    Di[np+1:np+ns,1:np,i]=Ai[:,:,i]
    Di[1:np,nt,i]=λ*alpha[:,i]'
    Di[nt,nt,i]=1
end

Ei=zeros(nt,nt,nt)
for i in 1:nt
    Ei[i,nt,i]=1
end
S0=Ei[:,:,np+1]

D_i=copy(Di)
for i in 1:ns
    D_i[ :, :,i] = (Di[ :, :,i] + Di[ :, :,i]') / 2
end
        
S_0 = (S0 + S0') / 2
E_i = copy(Ei)
for i in 1:nt
    E_i[ :, :,i] = (Ei[ :, :,i] + Ei[ :, :,i]') / 2
end

Cis=zeros(nt,nt,np)
for i in 1:np
    Cis[i,i,i]=1
end
#-----all matrices found



In [10]:
model = Model(SCS.Optimizer)
@variable(model, X[1:nt, 1:nt], PSD)
@objective(model, Min, tr(X * S_0))

for i=1:np
    @constraint(model, tr(X * Cis[:,:,i]) <=1.0)
end
for i=1:ns
    @constraint(model, tr(X * D_i[:,:,i]) == 0.0)
end
for i=1:nt
    @constraint(model, tr(X * E_i[:,:,i]) >= 0)
end

optimize!(model)
solution = value.(X)
println("Rank of the solution matrix X:")
println(rank(solution))


------------------------------------------------------------------
	       SCS v3.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 666, constraints m: 737
cones: 	  z: primal zero / dual free vars: 24
	  l: linear vars: 47
	  s: psd vars: 666, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 932, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 7.05e+00  4.98e-01  7.05e+00 -3.52e+00  1.00e-01  7.11e-04 
    25| 5.21e-05  5.35e-

In [11]:
model = Model(SCS.Optimizer)
@variable(model, X[1:nt, 1:nt], PSD)
@objective(model, Min, tr(X * S_0))

for i=1:np
    @constraint(model, tr(X * Cis[:,:,i]) <=1.0)
end
for i=1:ns
    @constraint(model, tr(X * D_i[:,:,i]) >= 0.0)
end
for i=1:ns
    @constraint(model, tr(X * D_i[:,:,i]) <= 0.0)
end
for i=1:nt
    @constraint(model, tr(X * E_i[:,:,i]) >= 0)
end

optimize!(model)
solution = value.(X)
println("Rank of the solution matrix X:")
println(rank(solution))


------------------------------------------------------------------
	       SCS v3.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 666, constraints m: 761
cones: 	  l: linear vars: 95
	  s: psd vars: 666, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1151, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 7.05e+00  4.98e-01  7.05e+00 -3.52e+00  1.00e-01  6.21e-04 
    25| 6.24e-05  8.94e-06  6.24e-05  3.12e-05  1.00e-01  3.07

In [12]:
model = Model(COSMO.Optimizer)

@variable(model, X[1:nt, 1:nt], PSD)
@objective(model, Min, tr(X * S_0))

for i = 1:np
    @constraint(model, tr(X * Cis[:, :, i]) <= 1.0)
end

for i = 1:ns
    @constraint(model, tr(X * D_i[:, :, i]) == 0.0)
end

for i = 1:nt
    @constraint(model, tr(X * E_i[:, :, i]) >= 0)
end

optimize!(model)
solution = value.(X)
println("Optimal X:")
# println(solution)

println("Rank of the solution matrix X:")
println(rank(solution))

------------------------------------------------------------------
          COSMO v0.8.9 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{666},
          constraints: A ∈ R^{737x666} (932 nnz),
          matrix size to factor: 1403x1403,
          Floating-point precision: Float64
Sets:     DensePsdConeTriangle of dim: 666 (36x36)
          Nonnegatives of dim: 36
          ZeroSet of dim: 24
          Box of dim: 11
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 5000,
          scaling iter = 10 (on),
          check termination every 25 iter,
          check infeasibility every 40 iter,
          KKT system solver: QDLDL
Acc:      Anderson Type2{QRDecomp},
          Memory size = 15, RestartedMemory,	
          S

In [13]:
model = Model(COSMO.Optimizer)

@variable(model, X[1:nt, 1:nt], PSD)
@objective(model, Min, tr(X * S_0))

for i = 1:np
    @constraint(model, tr(X * Cis[:, :, i]) <= 1.0)
end

for i = 1:ns
    @constraint(model, tr(X * D_i[:, :, i]) >= 0.0)
end

for i = 1:ns
    @constraint(model, tr(X * D_i[:, :, i]) <= 0.0)
end


for i = 1:nt
    @constraint(model, tr(X * E_i[:, :, i]) >= 0)
end

optimize!(model)
solution = value.(X)
println("Optimal X:")
# println(solution)

println("Rank of the solution matrix X:")
println(rank(solution))

------------------------------------------------------------------
          COSMO v0.8.9 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{666},
          constraints: A ∈ R^{761x666} (1151 nnz),
          matrix size to factor: 1427x1427,
          Floating-point precision: Float64
Sets:     DensePsdConeTriangle of dim: 666 (36x36)
          Nonnegatives of dim: 60
          Box of dim: 35
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 5000,
          scaling iter = 10 (on),
          check termination every 25 iter,
          check infeasibility every 40 iter,
          KKT system solver: QDLDL
Acc:      Anderson Type2{QRDecomp},
          Memory size = 15, RestartedMemory,	
          Safeguarded: true, tol: 2.0
S

# Try Solving the QCQP non-linearly?

In [14]:
model = Model(Ipopt.Optimizer)
@variable(model, Z[1:nt] >= 0)
@objective(model, Min, Z' * S_0 * Z)
for i in 1:np
    @constraint(model, Z' * Cis[:,:,i] * Z <= 1)
end
for i in 1:ns
    @constraint(model, Z' * D_i[:,:,i] * Z == 0.0)
end
for i=1:nt
    @constraint(model, Z' * E_i[:,:,i] * Z >= 0.0)
end
optimize!(model)
# solution_Z = value.(Z)
# objective_value = objective_value(model)

# println("Optimal solution Z: ", solution_Z)
# println("Objective value: ", objective_value)

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      414
Number of nonzeros in inequality constraint Jacobian.:       82
Number of nonzeros in Lagrangian Hessian.............:      267

Total number of variables............................:       36
                     variables with only lower bounds:       36
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       24
Total number of inequality constraints...............:       47
        inequality constraints with only lower bounds:       36
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       11

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999800e-05 1.00e-04 1.48e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

# How about solving the original optimisation problem